# Preprocessing Pipeline for Structural Data

- Sections:
    - Set-up: Run every time to create input dictionary
    - Part 1:
    - Part 2:
    - Part 3:

# Set-Up: 
## Select Subjects to Process and Set Variables - This sections needs to be run before each of the 3 Parts

### Packages and Functions Needed for This Notebook

In [1]:
import os
import shutil
from glob import glob
from nilearn import plotting
import subprocess
import sys
import matplotlib_inline
from tqdm import tqdm
import tempfile
from pathlib import Path


# export MPLBACKEND=TkAgg 

In [2]:
def submit_slurm_job(job_name, command, partition="bch-compute", nodes=1, cpus_per_task=16, mem="50G", time="24:00:00"):
    
    script = f"""#!/bin/bash
        #SBATCH --job-name={job_name}
        #SBATCH --partition={partition}
        #SBATCH --nodes={nodes}
        #SBATCH --cpus-per-task={cpus_per_task}
        #SBATCH --mem={mem}
        #SBATCH --time={time}

        # Run the command
        export MPLBACKEND=TkAgg
        
        {command}
        """
    
    # Write the script to a temporary file
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        f.write(script)
        script_file = f.name

    # Make the script executable
    subprocess.run(["chmod", "+x", script_file])

    try:
        # Submit the job using sbatch through the shell
        output = subprocess.check_output(['sbatch', script_file]).decode('utf-8')

        # Extract the job ID from the output
        job_id = output.strip().split()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error submitting job: {e}")
        job_id = None

    return job_id


In [3]:
def create_input_dict(input_folder, subjects_to_skip=None, input_type='FOLDER'):
    # Check if input_folder is a valid directory
    if not Path(input_folder).is_dir():
        raise ValueError("Input folder is not a valid directory")
    
    subject_sessions = {}
    
    if input_type=='BIDS':
        subjects=[f for f in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, f))]
        
        if subjects_to_skip is not None:
            subjects = [subject for subject in subjects if subject not in subjects_to_skip]
            
        print("There are", len(subjects), "unique subjects to be registered")
        
        for subject in sorted(subjects):
            # Get the path to the subject folder
            subject_path = os.path.join(input_dir, subject)

            # Get a list of session folders within the subject folder
            sessions = [f for f in os.listdir(subject_path) if os.path.isdir(os.path.join(subject_path, f))]

            # Add the subject and sessions to the dictionary
            subject_sessions[subject] = sessions


        print(subject_sessions)
        
    
    elif input_type=='FOLDER':
        #All selected scans are in one folder
        #Assumptions: first part of file name is subject ID followed by an _
        subjects=sorted(set([os.path.basename(i).split('_')[0] for i in glob(f'{input_dir}/*.nii*')]))
        
        if subjects_to_skip is not None:
            subjects = [subject for subject in subjects if subject not in subjects_to_skip]
            
        print("There are", len(subjects), "unique subjects to be registered")
        
        
        for subject in sorted(subjects):
            for file in glob(f'{input_dir}/*{subject}*.nii*'):
                subject = os.path.basename(file).split('_')[0]
                session = os.path.basename(file).split('_')[1]

                if subject not in subject_sessions:
                    subject_sessions[subject] = []
                if session not in subject_sessions[subject]:  
                    subject_sessions[subject].append(session)


        print(subject_sessions)

    
    else:
        raise ValueError(f"Invalid input_type: '{input_type}'. Should be either 'BIDS' or 'FOLDER'.")
    
    
    return subject_sessions

### Put your information below then create the input dictionary (subject_sessions)

In [15]:
input_dir="test_input_combine"  #Folder with input files
#input_dir = '/lab-share/Neuro-Cohen-e2/Public/lesions/RDCRN_TSC/'

input_type='BIDS' #'BIDS' or 'Folder'
subjects_to_skip=['sub-MGH083']  


output_dir="output_combine_test"


IMAGE_TYPES = ['T1w', 'T2w', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='T1w' #your ideal registration target, case sensitive
Reg_target_2='T2w' #your second choice for registration target, case sensitive

In [4]:
input_dir="RDCRN_test"  #Folder with input files
input_type='BIDS' #'BIDS' or 'Folder'
#input_dir = '/lab-share/Neuro-Cohen-e2/Public/lesions/RDCRN_TSC/'
output_dir="output_RDCDN_test_new_2"


IMAGE_TYPES = ['t1', 't2'] #case sensitive, change to match what you used e.g. t1, t1w, TI 
Reg_target_1='t1' #your ideal registration target, case sensitive
Reg_target_2='t2' #your second choice for registration target, case sensitive

#Bias correction only works on T1 or T2, bias correction is set to work on the registration target

In [5]:
subject_sessions=create_input_dict(input_dir, input_type='BIDS')

There are 2 unique subjects to be registered
{'7901-01-001': ['scan01', 'scan03', 'scan02'], '7901-01-002': ['scan01', 'scan02']}


# Part 1: Make Output Folders and Combine Images

## Part 1 Functions

In [16]:
def print_tree(d, n=5, indent=0):
    """
    Recursively prints the folder structure.
    
    Parameters:
    d (dict): The folder structure dictionary.
    indent (int): The indentation level (number of spaces).
    """
    
    subset = {k: subject_sessions[k] for k in list(subject_sessions)[:n]}
    
    for key, value in subset.items():
        print('    ' * indent + str(key))
        if isinstance(value, list):
            for item in value:
                print('    ' * (indent + 1) + str(item))
        elif isinstance(value, dict):
            print_tree(value, indent + 1)

In [6]:
def combine_images(working_dir, participant, session, image_type, list_of_images, clean_up=True):
    
    BIDSPATH = '/lab-share/Neuro-Cohen-e2/Public/lesions/MGH_Perinatal_Stroke_BIDS/code/bids_lesion_code'
   
    
    for i, image in enumerate(list_of_images, start=1):
        mask_file = f'{working_dir}/temp_{i}_{image_type}_mask.nii.gz'
        

        result = subprocess.run(['fslmaths', image, '-abs', '-bin', mask_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode != 0:
            print(result.stderr.decode())
            raise Exception(f'Failed to create mask for {image}')
        
    
    mask_files = [f'/app/data/temp_{i}_{image_type}_mask.nii.gz' for i in range(1, len(list_of_images) + 1)]
    output_file = f'/app/data/{participant}_{session}_{image_type}.nii.gz'
    
    cmd = [
        'singularity', 'exec',
        '-B', f'{working_dir}:/app/data',
        '-B', f'{BIDSPATH}:{BIDSPATH}',
        f'{BIDSPATH}/niftymic.sif',
        'niftymic_reconstruct_volume',
        '--filenames', *list_of_images,
        '--filenames-masks', *mask_files,
        '--output', output_file
    ]
    
    if clean_up == True:
            cmd += [
            'rm', '-r', 
            f'{working_dir}/config*', 
            f'{working_dir}/temp*', 
            f'{working_dir}/*mask*', 
            f'{working_dir}/motion_correction'
        ]
        
    command = ' '.join(cmd)

    return command


### Part 1a : Make output folder system

In [19]:
print(f'Your output folder system will look something like this: ')
print_tree(subject_sessions)

Your output folder system will look something like this: 
7901-01-001
    scan01
    scan03
    scan02
7901-01-002
    scan01
    scan02


In [7]:
combine_dict={}
for subject, sessions in subject_sessions.items():
    subject_folder = os.path.join(output_dir, subject)
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    
    for session in sorted(set(sessions)):
        print(f'Processing {session} for {subject}')
        session_folder = os.path.join(subject_folder, session)
         
        
        if os.path.exists(session_folder):
            if os.listdir(session_folder):
                print(f"{subject}: {session_folder} exists and is not empty")
        else:
            if not os.path.exists(session_folder):
                os.makedirs(session_folder)

Processing scan01 for 7901-01-001
Processing scan02 for 7901-01-001
Processing scan03 for 7901-01-001
Processing scan01 for 7901-01-002
Processing scan02 for 7901-01-002


### Part 1b: copy selected files to output folder system, combine images as needed (will submit to SLURM)

In [22]:
combine_dict={}
for subject, sessions in subject_sessions.items():
    subject_folder = os.path.join(output_dir, subject)

    for session in sorted(set(sessions)):
        session_folder = os.path.join(subject_folder, session)
         
        for image_type in IMAGE_TYPES:

            if glob(f'{session_folder}/*{image_type}*.nii*'):
                    print(f"Error: A file with image type '{image_type}' already exists in {session_folder}")
            else:    
                if input_type == 'FOLDER':
                    images = glob(f'{input_dir}/{subject}*{session}*{image_type}*.nii*')
                elif input_type == 'BIDS':
                    images = glob(f'{input_dir}/{subject}/{session}/*{image_type}*.nii*')
             
                if len(images) > 3:
                    print(f"Error: More than 3 images found for participant {subject}, session {session}, and image type {image_type}.")
                    continue
                if len(images) > 1:
                    print(f"combining: {images}")
                    command = combine_images(session_folder, subject, session, image_type, images, clean_up=True)
                    job_name = f"combine_images_{subject}_{session}_{image_type}"
                    job_id=submit_slurm_job(job_name, command)
                    combine_dict[(subject,session)] = job_id
                elif len(images) == 1:
                    print(f"moving: {images} to {session_folder}/{subject}_{session}_{image_type}.nii.gz")
                    shutil.copy(images[0], f'{session_folder}/{subject}_{session}_{image_type}.nii.gz')

if len(combine_dict) > 0:
    print('You have', len(combine_dict), 'Combine Jobs submitted to SLURM; subject and job IDs are stored in combine_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in combine_dict to see code outputs and any errors")
else:
    print('You have', len(combine_dict), 'Combine Jobs')

Error: A file with image type 't1' already exists in output_RDCDN_test_new_2/7901-01-001/scan01
Error: A file with image type 't2' already exists in output_RDCDN_test_new_2/7901-01-001/scan01
Error: A file with image type 't1' already exists in output_RDCDN_test_new_2/7901-01-001/scan02
Error: A file with image type 't2' already exists in output_RDCDN_test_new_2/7901-01-001/scan02
Error: A file with image type 't1' already exists in output_RDCDN_test_new_2/7901-01-001/scan03
Error: A file with image type 't2' already exists in output_RDCDN_test_new_2/7901-01-001/scan03
Error: A file with image type 't1' already exists in output_RDCDN_test_new_2/7901-01-002/scan01
Error: A file with image type 't2' already exists in output_RDCDN_test_new_2/7901-01-002/scan01
Error: A file with image type 't1' already exists in output_RDCDN_test_new_2/7901-01-002/scan02
Error: A file with image type 't2' already exists in output_RDCDN_test_new_2/7901-01-002/scan02
You have 0 Combine Jobs


# Part 2: Prepare Registration Images

## Part 2 Functions

In [27]:
def set_registration_target(file_names):
    global Reg_target_1
    global Reg_target_2
    reg_target = None
    for file_name in file_names:
        if Reg_target_1 in file_name:
            reg_target = Reg_target_1
            break
    if reg_target is None:
        for file_name in file_names:
            if Reg_target_2 in file_name:
                reg_target = Reg_target_2
                break
    if reg_target is None:
        raise ValueError(f"No registration target found in {file_names}")
    return reg_target

    
def reslice_image(input_folder, participant, session, reg_target):
    filename=f'{input_folder}/{participant}_{session}_{reg_target}.nii.gz'
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} does not exist")
    # Get the maximum pixel width
    cmd = f"fslinfo {filename} | grep pixdim[1-3] | awk '{{ print $2 }}' | sort -rn | head -1"
    max_pixelwidth = float(subprocess.check_output(cmd, shell=True).strip())

    if max_pixelwidth > 1.5:
        print(f"Largest pixel dimension is {max_pixelwidth} > 1.5mm, reslicing to 1mm isovolumetric")
        
        input_file = f"{input_folder}/{participant}_{session}_{reg_target}.nii.gz"
        size = 1
        output_file = f"{input_folder}/{participant}_{session}_{reg_target}_{size}mm.nii.gz"

        cmd = f"flirt -interp spline -in {input_file} -ref {input_file} -applyisoxfm {size} -out {output_file}"
        subprocess.run(cmd, shell=True)

        os.rename(input_file, f"{input_folder}/{participant}_{session}_{reg_target}_aniso.nii.gz")
        os.rename(output_file, input_file)
    else:
        print(f"Largest pixel dimension is {max_pixelwidth}, leaving image alone")
        

def bias_corr(input_folder, participant, session, reg_target, clean_up=True):
    stem = f'{input_folder}/{participant}_{session}_{reg_target}' 
    
    if os.path.exists(f"{stem}_orig.nii.gz"):
        print(f'{stem}_orig.nii.gz already exists, suggesting this image has been bias corrected already!')
        return
    
    if '1' in reg_target:
        img_type='T1'
    elif '2' in reg_target:
        img_type='T2'
        
    # Run fsl_anat_alt.sh
    cmd = f"./fsl_anat_alt.sh -i {stem} -t {img_type} --noreg --nosubcortseg --noseg"

        # Rename files
    cmd += f"mv {input_folder}/{participant}_{session}_{reg_target}.nii.gz {stem}_orig.nii.gz\n"
    cmd += f"mv {stem}.anat/T1_biascorr.nii.gz {stem}.nii.gz\n"

    # Run fslmaths
    cmd += f"fslmaths {stem}.nii.gz {stem}.nii.gz -odt short"
    
    if clean_up == True:
        cmd += f"rm -r {stem}.anat"
        
    return cmd
    

### Run Part 2; will submit jobs to SLURM

In [31]:
bias_corr_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(output_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=set_registration_target(glob(f'{session_folder}/*.nii*'))
        print(reg_target)
        
        print(f'Reslicing {reg_target}')
        reslice_image(session_folder, subject, session, reg_target)
        
        print(f'Bias Correcting {reg_target}')
        command=bias_corr(session_folder, subject, session, reg_target, clean_up=True) 
        job_name = f"bias_correct_{subject}_{session}_{reg_target}"
        job_id=submit_slurm_job(job_name, command)
        bias_corr_dict[(subject,session)] = job_id

if len(bias_corr_dict) > 0:
    print('You have', len(bias_corr_dict), 'Bias Correction Jobs submitted to SLURM; subject and job IDs are stored in bias_corr_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in bias_corr_dict to see code outputs and any errors')
else:
    print('You have', len(bias_corr_dict), 'Bias Correction Jobs')

*** Processing 7901-01-001: scan01 ***
t1
Reslicing t1
Largest pixel dimension is 1.0, leaving image alone
Bias Correcting t1
*** Processing 7901-01-001: scan03 ***
t1
Reslicing t1
Largest pixel dimension is 1.0, leaving image alone
Bias Correcting t1
*** Processing 7901-01-001: scan02 ***
t1
Reslicing t1
Largest pixel dimension is 1.0, leaving image alone
Bias Correcting t1
*** Processing 7901-01-002: scan01 ***
t1
Reslicing t1
Largest pixel dimension is 1.0, leaving image alone
Bias Correcting t1
*** Processing 7901-01-002: scan02 ***
t1
Reslicing t1
Largest pixel dimension is 1.0, leaving image alone
Bias Correcting t1
You have 5 Bias Correction Jobs submitted to SLURM; subject and job IDs are stored in bias_corr_dict
You can type "squeue -u $USER" into your terminal to track SLURM job progress
You can check the output file matching the jobid in bias_corr_dict to see code outputs and any errors


In [30]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            691069 bch-inter     bash ch236393  R    3:53:20      1 compute-10-13


# Part 3: Co-Register and Skull Strip Images

## Part 3 Functions

In [23]:
# Co-register files to the Registration Target
#consider using logging
def set_registration_target(file_names):
    global Reg_target_1
    global Reg_target_2
    reg_target = None
    for file_name in file_names:
        if Reg_target_1 in file_name:
            reg_target = Reg_target_1
            break
    if reg_target is None:
        for file_name in file_names:
            if Reg_target_2 in file_name:
                reg_target = Reg_target_2
                break
    if reg_target is None:
        raise ValueError(f"No registration target found in {file_names}")
    return reg_target

def co_register(working_dir, reg_image, moving_image, skullstrip=True, clean_up=True):
    
    if not os.path.exists(f'{working_dir}/warps'):
        os.makedirs(f'{working_dir}/warps')
    
    moving_stem=os.path.basename(moving_image).split('.')[0]
    
    if os.path.exists(f"{working_dir}/{moving_stem}_space-{reg_target}.nii.gz"):
        print(f"WARNING: Input image file {moving_stem}_space-{reg_target}.nii.gz already exists. Skipping...")
        return 
    
    cmd = f"conda run -n nimlab_py310_2023_11 antsRegistrationSyNQuick.sh -d 3 -m {moving_image} -f {reg_image} -t sr -o {working_dir}/warps/{moving_stem}_space-{reg_target}"
    
    cmd +=f"mv {working_dir}/warps/{moving_stem}_space-{reg_target}Warped.nii.gz {working_dir}/{moving_stem}_space-{reg_target}.nii.gz"
    
    if clean_up == True:
        cmd +=f" rm -r {working_dir}/warps"
        
    
    if skullstrip=True:
        filename=f'{moving_stem}_space-{reg_target}'
        out_file=f'{session_folder}/COREGISTERED/{filename}_SkullStripped.nii.gz'
        out_mask=f'{session_folder}/COREGISTERED/{filename}_brain-mask.nii.gz'
        cmd += f'mri_synthstrip -i {file} -o {out_file} -m {out_mask}'
       
        
    # try:
    #     output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # except subprocess.CalledProcessError as e:
    #     print(f"Error: command '{e.cmd}' failed with exit code {e.returncode}")
    #     print(f"Output: {e.output.decode('utf-8')}")
    #     print(f"Error message: {e.stderr.decode('utf-8')}")
    #     return
        
    return cmd
        
    # remove non-BIDS suffix
    # src = f"{working_dir}/warps/{moving_stem}_space-{reg_target}Warped.nii.gz"
    # dst = f"{working_dir}/{moving_stem}_space-{reg_target}.nii.gz"
    # os.rename(src, dst)
    
    # shutil.rmtree(f'{working_dir}/warps/')
    


### Run Part 3

In [ ]:
co_reg_dict={}
for subject, sessions in subject_sessions.items():
    reg_target=None
    subject_folder = os.path.join(output_dir, subject) 
    for session in sessions:
        print(f'*** Processing {subject}: {session} ***')
        session_folder=os.path.join(subject_folder, session)
        
        reg_target=set_registration_target(glob(f'{session_folder}/*.nii*'))
        print(reg_target)
        
        print(f'Registering Images to the {reg_target}') #~5 min per file
        if not os.path.exists(f'{session_folder}/COREGISTERED'):
            os.makedirs(f'{session_folder}/COREGISTERED')
        reg_file=f'{session_folder}/{subject}_{session}_{reg_target}.nii.gz'
        for file in glob(f'{session_folder}/*.nii*'):
        #for file in tqdm(glob(f'{session_folder}/*.nii*'), desc='Registering images'):
                if reg_target not in file:
                    print(f'Regestering {file}')
                    command=co_register(f'{session_folder}/COREGISTERED', reg_file, file, skullstrip=True)
                    job_name = f"co-register_{subject}_{session}_{reg_target}"
                    job_id=submit_slurm_job(job_name, command)
                    co_reg_dict[(subject,session)] = job_id

if len(co_reg_dict) > 0:
    print('You have', len(co_reg_dict), 'Co-Registration Jobs submitted to SLURM; subject and job IDs are stored in the co_reg_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in co_reg_dict to see code outputs and any errors")
                                     
        
#         #Skull Strip
#         for file in glob(f'{session_folder}/COREGISTERED/*.nii*'):
#             filename=os.path.basename(file).split('.')[0]
#             out_file=f'{session_folder}/COREGISTERED/{filename}_SkullStripped.nii.gz'
#             out_mask=f'{session_folder}/COREGISTERED/{filename}_brain-mask.nii.gz'
#             command = f'mri_synthstrip -i {file} -o {out_file} -m {out_mask}'
            
#             print(f"Processing file: {file}")
#             try:
#                 subprocess.run(command, shell=True, check=True)
#             except subprocess.CalledProcessError as e:
#                 print(f"Error running mri_synthstrip: {e}")
        
        
        

In [ ]:
#check brain masks to catch really wacky co-reg - like dice of brain masks or something
#visualize